## 1. Video input

In [17]:
import cv2
import mediapipe as mp
import pyautogui

# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)
mp_draw = mp.solutions.drawing_utils

# Open webcam
cap = cv2.VideoCapture(0)

# To avoid multiple triggers
last_gesture = None

while True:
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    h, w, c = frame.shape
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    gesture = "None"

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Wrist position
            wrist = hand_landmarks.landmark[0]
            wrist_x, wrist_y = int(wrist.x * w), int(wrist.y * h)

            # Frame center
            center_x, center_y = w // 2, h // 2
            offset_x, offset_y = 100, 80  # tolerance

            if wrist_x < center_x - offset_x:
                gesture = "LEFT"
            elif wrist_x > center_x + offset_x:
                gesture = "RIGHT"
            elif wrist_y < center_y - offset_y:
                gesture = "UP"
            elif wrist_y > center_y + offset_y:
                gesture = "DOWN"
            else:
                gesture = "CENTER"

            # Draw markers
            cv2.circle(frame, (wrist_x, wrist_y), 10, (0, 255, 0), cv2.FILLED)
            cv2.circle(frame, (center_x, center_y), 10, (255, 0, 0), cv2.FILLED)

            # Show gesture text
            cv2.putText(frame, f"Gesture: {gesture}", (10, 50),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 3)

            # Map gestures to keyboard keys
            if gesture != last_gesture:  # avoid repeats
                if gesture == "LEFT":
                    pyautogui.press("left")
                elif gesture == "RIGHT":
                    pyautogui.press("right")
                elif gesture == "UP":
                    pyautogui.press("up")
                elif gesture == "DOWN":
                    pyautogui.press("down")
                last_gesture = gesture

    cv2.imshow("Hand Gesture Control", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Gesture recognition